In [1]:
import numpy as np
#import pandas as pd

from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, Flatten, Dense, Input, ReLU, Dropout
import tensorflow.keras as keras
from tensorflow.keras import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from PIL import Image

from ultralytics import YOLO

In [2]:
#import data
#read the data
#The data is read from the directory and each batch can be thought of as a set of labels and datapoints
#Might need to change the directory depending on where

#50
train = keras.utils.image_dataset_from_directory(
    directory='../../data/Faces',
    labels='inferred',
    label_mode='categorical',
    batch_size=140,
    image_size=(1000, 1000),
    validation_split = 0.2,
    subset = 'training',
    seed = 50
    )

validation = keras.utils.image_dataset_from_directory(
    directory='../../data/Faces',
    labels='inferred',
    label_mode='categorical',
    batch_size=140,
    image_size=(1000, 1000),
    validation_split = 0.2,
    subset = 'validation',
    seed = 50
    )

Found 140 files belonging to 6 classes.
Using 112 files for training.


Found 140 files belonging to 6 classes.
Using 28 files for validation.


In [3]:
#convert images to numpy
train_processed_numpy = train.as_numpy_iterator()
validation_processed_numpy = validation.as_numpy_iterator()

X_train, y_train = train_processed_numpy.next()
X_val, y_val = validation_processed_numpy.next()

In [4]:
#convert the images to cropped faces
face_classifier = YOLO("../../app/src/face_detection/model/yolov8n-face.pt")

X_cropped_train = []
y_train_cropped = []

for i in range(112):
    face = X_train[i]
    faces = face_classifier.predict(face)
    boxes = faces[0].boxes.xyxy.tolist()
    if boxes:
        
        #extract the face based on the output from the YOLOv8 model        
        left, bottom, right, top = boxes[0]
        cropped_face = face[int(bottom):int(top), int(left):int(right)]
        #resize cropped face to a std shape, 100x100 for now but can adjust this
        pil_face = Image.fromarray(np.uint8(cropped_face))
        pil_face = pil_face.resize((48, 48))
        numpy_cropped_face = np.array(pil_face)
        #append this to the new list containing all cropped faces
        X_cropped_train.append(numpy_cropped_face)
        y_train_cropped.append(y_train[i])

X_val_cropped = []
y_val_cropped = []

for i in range(28):
    face = X_val[i]
    faces = face_classifier.predict(face)
    boxes = faces[0].boxes.xyxy.tolist()
    if boxes:
        
        #extract the face based on the output from the YOLOv8 model        
        left, bottom, right, top = boxes[0]
        cropped_face = face[int(bottom):int(top), int(left):int(right)]
        #resize cropped face to a std shape, 100x100 for now but can adjust this
        pil_face = Image.fromarray(np.uint8(cropped_face))
        pil_face = pil_face.resize((48, 48))
        numpy_cropped_face = np.array(pil_face)
        #append this to the new list containing all cropped faces
        X_val_cropped.append(numpy_cropped_face)
        y_val_cropped.append(y_val[i])


c:\Users\veena\OneDrive\Desktop\Nus\Y4S2\IS4242\Project Actual\moodify\emotion_detection\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0: 640x640 1 face, 171.8ms
Speed: 19.0ms preprocess, 171.8ms inference, 2331.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 117.5ms
Speed: 16.0ms preprocess, 117.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 111.9ms
Speed: 11.0ms preprocess, 111.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 119.6ms
Speed: 11.7ms preprocess, 119.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 175.2ms
Speed: 14.0ms preprocess, 175.2ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 156.5ms
Speed: 17.0ms preprocess, 156.5ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 211.4ms
Speed: 15.0ms preprocess, 211.4ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 165.1ms
Speed: 16.2ms preprocess, 165.1ms inference, 2.0ms postprocess per image at shape

In [5]:
#convert data into neural net input format

X_cropped_train = np.array(X_cropped_train)
y_train_cropped = np.array(y_train_cropped)
X_val_cropped = np.array(X_val_cropped)
y_val_cropped = np.array(y_val_cropped)

In [6]:
#first convolutional layer
#source of archi: https://cs230.stanford.edu/projects_winter_2020/reports/32610274.pdf

default_filters = 64

input_x = Input(shape = (48, 48, 3))

conv_1 = Conv2D(filters = default_filters, kernel_size = (3, 3), name = 'conv_1', padding = 'same',activation='relu' ,data_format='channels_last')(input_x)
batch_norm1 = BatchNormalization(name = 'batch_norm1')(conv_1)
conv_12 = Conv2D(filters = default_filters, kernel_size = (3, 3), name = 'conv_11', padding = 'same', activation = 'relu')(batch_norm1)#max_pooling
batch_norm12 = BatchNormalization(name = 'batch_norm12')(conv_12)
max_pool1 = MaxPool2D(name = 'max_pool1', pool_size = (2,2), strides = (2,2))(batch_norm12)
drop_out1 = Dropout(0.5)(max_pool1)

#second convolutional layer 
conv2 = Conv2D(filters = 2 * default_filters, kernel_size = (3,3), name = 'conv_2', padding = 'same', activation = 'relu')(drop_out1)
batch_norm2 = BatchNormalization(name = 'batch_norm2')(conv2)
conv21 = Conv2D(filters = 2 * default_filters, kernel_size = (3,3), name = 'conv_21', padding = 'same', activation = 'relu')(conv2)
batch_norm21 = BatchNormalization(name = 'batch_norm21')(conv21)
max_pool2 = MaxPool2D(name = 'max_pool2', pool_size = (2,2), strides = (2,2))(batch_norm21)
dropout_2 = Dropout(0.5)(max_pool2)




#third convolutional layer
conv3 = Conv2D(filters = 2 * 2 * default_filters, kernel_size = (3,3), name = 'conv_3', padding = 'same', activation = 'relu')(dropout_2)
batch_norm3 = BatchNormalization(name = 'batch_norm3')(conv3)
conv31 = Conv2D(filters = 2 * 2 * default_filters, kernel_size = (3,3), name = 'conv_31', padding = 'same', activation = 'relu')(batch_norm3)
batch_norm31 = BatchNormalization(name = 'batch_norm31')(conv31)
max_pool3 = MaxPool2D(name = 'max_pool3', pool_size = (2,2), strides = (2,2))(batch_norm31)
dropout_3 = Dropout(0.5)(max_pool3)

#fourth convolutional layer
conv4 = Conv2D(filters = 2 * 2 * 2 * default_filters, kernel_size = (3,3), name = 'conv_4', padding = 'same', activation = 'relu')(dropout_3)
batch_norm4 = BatchNormalization(name = 'batch_norm4')(conv4)
conv41 = Conv2D(filters = 2 *2 * 2 * default_filters, kernel_size = (3,3), name = 'conv_41', padding = 'same', activation = 'relu')(batch_norm4)
batch_norm41 = BatchNormalization(name = 'batch_norm41')(conv41)
max_pool4 = MaxPool2D(name = 'max_pool4', pool_size = (2,2), strides = (2,2))(batch_norm41)
dropout_4 = Dropout(0.5)(max_pool4)
#dense layer

flat = Flatten()(dropout_4)

dense1 = Dense(256, activation = 'relu', name = 'dense_1')(flat)
drop_out_dense1 = Dropout(0.4)(dense1)
dense2 = Dense(128, activation = 'relu', name = 'dense_2')(drop_out_dense1)
drop_out_dense2 = Dropout(0.4)(dense2)
#final softmax layer
dense3 = Dense(64, activation = 'relu', name = 'dense_3')(drop_out_dense2)
drop_out_dense3 = Dropout(0.5)(dense3)

#6 outputs for the 6 classes
output = Dense(6, activation = 'softmax', name = 'output')(drop_out_dense3)


# In[11]:

In [7]:
checkpoint = ModelCheckpoint(filepath = 'cnn_scratch.keras')


# In[ ]:


model = Model(input_x, output)

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['acc'])

model.fit(x = X_cropped_train/255, y = y_train_cropped, epochs = 100, validation_data = (X_val_cropped/255, y_val_cropped), callbacks = [checkpoint])

Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 384ms/step - acc: 0.1290 - loss: 6.0435 - val_acc: 0.1786 - val_loss: 1.7935
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 301ms/step - acc: 0.1707 - loss: 4.7397 - val_acc: 0.1786 - val_loss: 1.7921
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 303ms/step - acc: 0.1712 - loss: 3.5111 - val_acc: 0.1786 - val_loss: 1.7921
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 301ms/step - acc: 0.2039 - loss: 3.0482 - val_acc: 0.1786 - val_loss: 1.7952
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 436ms/step - acc: 0.2518 - loss: 3.0347 - val_acc: 0.2143 - val_loss: 1.7952
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 685ms/step - acc: 0.2211 - loss: 2.7043 - val_acc: 0.2143 - val_loss: 1.7912
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 588ms/step - acc: 0.1691 - loss: 2.6979 - val_acc: 0.1071 - val_loss: 1.7900
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 551ms/step - acc: 0.1811 - loss: 2.5625 - val_acc: 0.1786 - val_loss: 1.7920
Epoch 9/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 554ms/step - acc: 0.1826